In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib
from matplotlib import pyplot as plt
from textwrap import wrap


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

import pickle
import time
from tqdm import tqdm

from itertools import product

#import googletrans
#from googletrans import Translator
#to avoid instantiate too many translator, we created one instance for the whole notebook
#translator = Translator()


from prophet import Prophet
import itertools
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import holidays


import hts.functions
import collections
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree


from sktime.transformations.hierarchical.reconcile import Reconciler
from sktime.transformations.hierarchical.aggregate import Aggregator
from sktime.forecasting.exp_smoothing import ExponentialSmoothing


import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

# define functions

In [2]:
def get_best_hyperparams(data):   
    
    default_best_params = {'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.5}
    
    first_cutoff_date = pd.to_datetime('2014-02-1')
    first_nonzero_date = data['y'].ne(0).idxmax()
    
    # return default_best_params if 
    # 1. all values are zero
    # 2. too many values are zero
    if (data['y'] == 0).all() or (first_cutoff_date < first_nonzero_date):
        print( first_nonzero_date)
        print("too many zero values, return default best params")
        return default_best_params
    else:
        param_grid = {  
                'changepoint_prior_scale': [0.01,0.2],
                'seasonality_prior_scale': [0.01,0.5 ],
        }

        # Generate all combinations of parameters
        all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
        rmses = []  # Store the RMSEs for each params here

        # Use cross validation to evaluate all parameters
        cutoffs = pd.to_datetime(['2014-2-1', '2015-6-1'])

        for params in all_params:
            m = Prophet(**params).fit(data)  # Fit model with given params
            df_cv = cross_validation(m, cutoffs=cutoffs, horizon='90 days', parallel="processes")
            df_p = performance_metrics(df_cv, rolling_window=1)
            rmses.append(df_p['rmse'].values[0])

        # Find the best parameters
        tuning_results = pd.DataFrame(all_params)
        tuning_results['rmse'] = rmses

        best_params = all_params[np.argmin(rmses)]
        print(best_params)
        return best_params


In [3]:
def get_best_preds(df_,   #df_heir_train
                   cat_id  #cat_id in oneshop_cat_list 
                  ):
    #get dataframe
    columns = ['ds', cat_id]
    df = df_.loc[:, columns]
    df.columns = ['ds', 'y']
    
    #get best_params
    best_params = get_best_hyperparams(df)
    
    #create model based on best_params
    m = Prophet(changepoint_prior_scale=best_params['changepoint_prior_scale'], 
                seasonality_prior_scale=best_params['seasonality_prior_scale'], 
                yearly_seasonality=True)
    m.add_country_holidays(country_name='RU')
    m.fit(df)

    #predict and return
    future = m.make_future_dataframe(periods=1, freq='MS')
    forecast = m.predict(future)

    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(3))
    #fig1 = m.plot(forecast)

    best_preds = forecast.tail(1).yhat.values[0]
    print('best_preds: ', best_preds)
    return best_preds
    

# import data from pickle file

In [4]:
pickle_dict2 = pickle.load(open('../temp/data2.pkl', 'rb'))
df_basegrid = pickle_dict2['df_basegrid']   #concat df_train_m and df_test


In [5]:
list_shop_id=sorted(df_basegrid['shop_id'].unique())
print(list_shop_id)
print(len(list_shop_id))

[2, 3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59]
42


### shop_id: 36 only have one month data, won't be feed into forcasting

In [6]:
list_shop_id_filtered = list_shop_id.copy()
list_shop_id_filtered.remove(36)
print(list_shop_id_filtered)
print(len(list_shop_id_filtered))

[2, 3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 34, 35, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59]
41


### forecasting all category sales in all shops

In [8]:
%%time
    
for shop_id in tqdm(list_shop_id_filtered):
    
    print(f'\n\nforecasting shop: s{shop_id}')
    
    ####read pickle_dict, which store everything
    #############################################
    pickle_dict_oneshop = pickle.load(open(f'../temp/s{shop_id}_hier.pkl', 'rb'))
    df_hier  = pickle_dict_oneshop['df_hier']
    tree = pickle_dict_oneshop['tree']
    sum_mat = pickle_dict_oneshop['sum_mat']
    sum_mat_labels =pickle_dict_oneshop['sum_mat_labels']
    oneshop_cat_list = pickle_dict_oneshop['oneshop_cat_list']
    oneshop_item_list = pickle_dict_oneshop['oneshop_item_list']


    ################### creat train, validation,and test dataset
    ###########################################################################
    df_hier_train = df_hier.loc[df_hier.index <= '2015-10-01']
    df_hier_test = df_hier.loc[df_hier.index == '2015-11-01']


    
    forecasts = pd.DataFrame(columns = df_hier.columns, index=['fake'])
    
    
    ################### forecasting total sales in this shop
    #################################################################
    print('\ngetting best preds for forecast_total_sales')
    forecasts['total'] = [get_best_preds(df_hier_train, 'total')]

    
  
    ################### forecasting all category sales in this shop
    #################################################################
    #dataframe to hold the forecasts
    for cat in oneshop_cat_list:
        print(f'\ngetting best preds for s{shop_id}_', cat)
        forecasts[cat] = [get_best_preds(df_hier_train, cat)]


    ###########save neccessary object to pickle_dict, 
    ############################################################################
    pickle_dict_oneshop_new = dict()
    pickle_dict_oneshop_new['forecasts']=forecasts
    pickle.dump(pickle_dict_oneshop_new, open(f'../temp/s{shop_id}_forecasts.pkl', 'wb'))



  0%|                                                                                           | 0/41 [00:00<?, ?it/s]



forecasting shop: s2

getting best preds for forecast_total_sales
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds     yhat  yhat_lower  yhat_upper
32 2015-09-01   838.49      739.63      943.33
33 2015-10-01   803.44      703.19      912.22
34 2015-11-01 1,133.29    1,034.19    1,235.51
best_preds:  1133.2869919017853

getting best preds for s2_ c0
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s2_ c11
2014-12-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.14       -0.21        0.52
33 2015-10-01  0.13       -0.24        0.48
34 2015-11-01  0.20       -0.15        0.58
best_preds:  0.19707196267481153

getting best preds for s2_ c12
{'changepoi

{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 45.31       40.30       50.12
33 2015-10-01 62.51       57.53       67.16
34 2015-11-01 49.49       44.82       54.58
best_preds:  49.487794307098646

getting best preds for s2_ c41
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  2.25        0.49        3.94
33 2015-10-01  3.57        1.93        5.17
34 2015-11-01  4.16        2.44        5.91
best_preds:  4.161186896000797

getting best preds for s2_ c42
2015-05-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  8.65        7.73        9.58
33 2015-10-01  8.72        7.85        9.60
34 2015-11-01 10.82        9.85       11.65
best_preds:  10.816719657063837

getting best preds for s2_ c43
{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.01}
           ds  

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  3.14        1.36        5.13
33 2015-10-01  3.22        1.31        5.12
34 2015-11-01  2.09        0.21        4.09
best_preds:  2.0862043873307967

getting best preds for s2_ c74
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s2_ c75
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  6.39        3.11        9.55
33 2015-10-01  6.15        3.00        9.27
34 2015-11-01  6.34        3.24        9.38
best_preds:  6.343388713166685

getting best preds for s2_ c76
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat

  2%|█▉                                                                              | 1/41 [14:53<9:55:30, 893.26s/it]

...



forecasting shop: s59

getting best preds for forecast_total_sales
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds     yhat  yhat_lower  yhat_upper
32 2015-09-01   919.60      852.40      986.34
33 2015-10-01   880.92      808.03      949.75
34 2015-11-01 1,442.94    1,372.86    1,516.26
best_preds:  1442.9427609109266

getting best preds for s59_ c0
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s59_ c11
2015-05-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.04       -0.11        0.19
33 2015-10-01  0.05       -0.10        0.19
34 2015-11-01  0.10       -0.04        0.25
best_preds:  0.09601745665259093

getting best preds for s59_ c12
{'chang

{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 75.92       51.47       98.18
33 2015-10-01 77.47       54.01       99.99
34 2015-11-01 81.16       56.58      104.25
best_preds:  81.15772730155213

getting best preds for s59_ c41
{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  7.10        3.66       10.81
33 2015-10-01  7.45        3.93       10.78
34 2015-11-01  7.51        3.93       11.00
best_preds:  7.505029231553532

getting best preds for s59_ c42
2015-06-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  4.37        3.90        4.84
33 2015-10-01  3.54        3.11        4.00
34 2015-11-01  4.88        4.41        5.36
best_preds:  4.883998142275225

getting best preds for s59_ c43
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           d

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  2.16        0.06        4.44
33 2015-10-01  2.18       -0.04        4.50
34 2015-11-01  2.09       -0.14        4.38
best_preds:  2.085178554280344

getting best preds for s59_ c74
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s59_ c75
{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 11.52        9.66       13.27
33 2015-10-01 12.06       10.33       13.73
34 2015-11-01 27.02       25.26       28.87
best_preds:  27.02240871480071

getting best preds for s59_ c76
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yha

100%|██████████████████████████████████████████████████████████████████████████████| 41/41 [10:32:00<00:00, 924.90s/it]

Wall time: 10h 32min
